This is the third and final tutorial on doing “NLP From Scratch”, where we write our own classes and functions to preprocess the data to do our NLP modeling tasks. We hope after you complete this tutorial that you’ll proceed to learn how torchtext can handle much of this preprocessing for you in the three tutorials immediately following this one.

In this project we will be teaching a neural network to translate from French to English.

This is made possible by the simple but powerful idea of the sequence to sequence network, in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a vector, and a decoder network unfolds that vector into a new sequence.

To improve upon this model we’ll use an attention mechanism, which lets the decoder learn to focus over a specific range of the input sequence.

**Read about sequence to sequence Neural networks**

In [64]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import time
import math

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as fxnl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger. We will however cheat a bit and trim the data to only use a few thousand words per language.


We’ll need a unique index per word to use as the inputs and targets of the networks later. To keep track of all this we will use a helper class called Lang which has word → index (word2index) and index → word (index2word) dictionaries, as well as a count of each word word2count to use to later replace rare words.

In [2]:
SOS_token = 0
EOS_token = 1

In [3]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word_to_index = {}
        self.word_to_count = {}
        self.index_to_word = {0: "SOS", 1: "EOS"}
        self.num_words = 2
    
    def add_Sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_Word(word)
    
    def add_Word(self, word):
        if word not in self.word_to_index:
            self.word_to_index[word] = self.num_words
            self.word_to_count[word] = 1
            self.index_to_word[self.num_words] = word
            self.num_words += 1
        else:
            self.word_to_count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [4]:
def unicode_to_ascii(unicode_str):
    return ''.join(char for char in unicodedata.normalize('NFD', unicode_str) 
    if unicodedata.category(char) != 'Mn')

def normalize_String(str_var):
    str_var = unicode_to_ascii(str_var.lower().strip())
    str_var = re.sub(r"([.!?])", r" \1", str_var)
    str_var = re.sub(r"[^a-zA-Z.!?]+", r" ", str_var)
    return str_var

To read the data file we will split the file into lines, and then split lines into pairs. The files are all English → Other Language, so if we want to translate from Other Language → English I added the reverse flag to reverse the pairs.

In [5]:
def read_Langs(language_one, language_two, reverse=False):
    print("Reading lines....... ")

    lines = open('dataset/%s-%s.txt' % (language_one, language_two), encoding='utf-8').\
        read().strip().split('\n')
    
    pairs = [[normalize_String(s) for s in l.split('\t')] for l in lines]

    if reverse:
        pairs = [list(reversed(pair)) for pair in pairs]
        input_language = Lang(language_two)
        output_language = Lang(language_one)
    else:
        input_language = Lang(language_one)
        output_language = Lang(language_two)
    
    return input_language, output_language, pairs

Since there are a lot of example sentences and we want to train something quickly, we’ll trim the data set to only relatively short and simple sentences. Here the maximum length is 10 words (that includes ending punctuation) and we’re filtering to sentences that translate to the form “I am” or “He is” etc. (accounting for apostrophes replaced earlier).

In [6]:
MAX_LENGTH = 10
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

In [7]:
def filter_Pair(pair):
    return len(pair[0].split(' ')) < MAX_LENGTH and \
        len(pair[1].split(' ')) < MAX_LENGTH and \
        pair[1].startswith(eng_prefixes)

In [8]:
def filter_Pairs(pairs): return [pair for pair in pairs if filter_Pair(pair)]

The full process for preparing the data is:

    * Read text file and split into lines, split lines into pairs
    * Normalize text, filter by length and content
    * Make word lists from sentences in pairs

In [9]:
def prepare_Data(language_one, language_two, reverse=False):
    input_language, output_language, pairs = read_Langs(language_one=language_one,     language_two=language_two, reverse=reverse)

    print("+" * 89)
    print("Read %s sentence pairs" % len(pairs))
    print("+" * 89)

    pairs = filter_Pairs(pairs)

    print("Trimmed to %s sentence pairs" % len(pairs))
    print("+" * 89)
    print("Counting words...")
    print("+" * 89)

    for pair in pairs:
        input_language.add_Sentence(pair[0])
        output_language.add_Sentence(pair[1])
    
    print("Counted words:")
    print("-" * 89)
    print(input_language.name, "->", input_language.num_words)
    print("-" * 89)
    print(output_language.name, "->", output_language.num_words)

    return input_language, output_language, pairs


In [10]:
input_language, output_language, pairs = prepare_Data("eng", "fra", True)
print("-" * 89)
print(input_language, "\n")
print("-" * 89)
print(output_language, "\n")
print("-" * 89)
print(pairs, "\n")
print("-" * 89)
print(random.choice(pairs))

ique .', 'he is in excellent physical condition .'], ['il projette de lancer son entreprise .', 'he is planning to launch his business .'], ['il est cale en litterature anglaise .', 'he is well read in english literature .'], ['il travaille dans le domaine de la biologie .', 'he is working in the field of biology .'], ['il est un celebre chanteur populaire au japon .', 'he s a famous popular singer in japan .'], ['c est une pointure dans son domaine .', 'he s a leading authority in his field .'], ['c est une sommite dans son domaine .', 'he s a leading authority in his field .'], ['il est etudiant en litterature japonaise .', 'he s a student of japanese literature .'], ['c est un jeune adolescent impressionnable .', 'he s a young impressionable teenager .'], ['il depend totalement de ses parents .', 'he s totally dependent on his parents .'], ['je suis libre jusqu a heures ce soir .', 'i am free till o clock this evening .'], ['je vais ecrire une lettre demain .', 'i am going to write 

A Recurrent Neural Network, or RNN, is a network that operates on a sequence and uses its own output as input for subsequent steps.

A Sequence to Sequence network, or seq2seq network, or Encoder Decoder network, is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.



Unlike sequence prediction with a single RNN, where every input corresponds to an output, the seq2seq model frees us from sequence length and order, which makes it ideal for translation between two languages.

Consider the sentence “Je ne suis pas le chat noir” → “I am not the black cat”. Most of the words in the input sentence have a direct translation in the output sentence, but are in slightly different orders, e.g. “chat noir” and “black cat”. Because of the “ne/pas” construction there is also one more word in the input sentence. It would be difficult to produce a correct translation directly from the sequence of input words.

With a seq2seq model the encoder creates a single vector which, in the ideal case, encodes the “meaning” of the input sequence into a single vector — a single point in some N dimensional space of sentences.



The Encoder
The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size,  hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input_tensor, hidden_tensor):
        embedded = self.embedding(input_tensor).view(1, 1, -1)
        output_tensor = embedded
        output_tensor, hidden_tensor = self.gru(output_tensor, hidden_tensor)
        return output_tensor, hidden_tensor
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder

The decoder is another RNN that takes the encoder output vector(s) and outputs a sequence of words to create the translation.

Simple Decoder
In the simplest seq2seq decoder we use only last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string <SOS> token, and the first hidden state is the context vector (the encoder’s last hidden state).

In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()

        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input_tensor, hidden_tensor):
        output_tensor = self.embedding(input_tensor).view(1, 1, -1)
        output_tensor = fxnl.relu(output_tensor)
        output_tensor, hidden_tensor = self.gru(output_tensor, hidden_tensor)
        output_tensor = self.softmax(self.out(output_tensor[0]))
        return output_tensor, hidden_tensor

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to save space we’ll be going straight for the gold and introducing the Attention Mechanism.

Attention Decoder
If only the context vector is passed betweeen the encoder and decoder, that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to “focus” on a different part of the encoder’s outputs for every step of the decoder’s own outputs. First we calculate a set of attention weights. These will be multiplied by the encoder output vectors to create a weighted combination. The result (called attn_applied in the code) should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.


Calculating the attention weights is done with another feed-forward layer attn, using the decoder’s input and hidden state as inputs. Because there are sentences of all sizes in the training data, to actually create and train this layer we have to choose a maximum sentence length (input length, for encoder outputs) that it can apply to. Sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.

In [13]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = fxnl.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = fxnl.relu(output)
        output, hidden = self.gru(output, hidden)

        output = fxnl.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training

Preparing Training Data

To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). While creating these vectors we will append the EOS token to both sequences.

In [14]:
def indexes_from_Sentence(lanuage, sentence):    
    return [lanuage.word_to_index[word] for word in sentence.split(' ')]

def tensor_from_Sentence(lanuage, sentence):
    indexes = indexes_from_Sentence(lanuage=lanuage, sentence=sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensor_from_Pair(pair):
    input_tensor = tensor_from_Sentence(input_language, pair[0])
    target_tensor = tensor_from_Sentence(output_language, pair[1])
    return (input_tensor, target_tensor)

Training the Model

To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. Then the decoder is given the <SOS> token as its first input, and the last hidden state of the encoder as its first hidden state.

“Teacher forcing” is the concept of using the real target outputs as each next input, instead of using the decoder’s guess as the next input. Using teacher forcing causes it to converge faster but when the trained network is exploited, it may exhibit instability.

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - intuitively it has learned to represent the output grammar and can “pick up” the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.

Because of the freedom PyTorch’s autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. Turn teacher_forcing_ratio up to use more of it.

In [15]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for e_idx in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[e_idx], encoder_hidden)
        encoder_outputs[e_idx] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if  random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for d_idx in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            loss += criterion(decoder_output, target_tensor[d_idx])
            decoder_input = target_tensor[d_idx]
    
    else:
        for d_idx in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[d_idx])
            if decoder_input.item() == EOS_token: break

    
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length



In [16]:
def as_Minutes(seconds):
    min = math.floor(seconds / 60)
    seconds -= min * 60
    return '%dm %ds' % (min, seconds)

def time_since(since, percent):
    now = time.time()
    seconds = now - since
    es = seconds / (percent)
    rs = es - seconds
    return '%s (- %s)' % (as_Minutes(seconds), as_Minutes(rs))

The whole training process looks like this:


Start a timer
Initialize optimizers and criterion
Create set of training pairs
Start empty losses array for plotting
Then we call train many times and occasionally print the progress (% of examples, time so far, estimated time) and average loss.

In [17]:
def train_Iters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0 
    plot_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensor_from_Pair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor=input_tensor, target_tensor=target_tensor, encoder=encoder,
        decoder=decoder, encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
        criterion=criterion)

        print_loss_total += loss
        plot_loss_total += loss


        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0 
            print('%s (%d %d%%) %.4f' % (time_since(start, iter / n_iters), iter, 
            iter / n_iters * 100, print_loss_avg))
        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0 
    
    show_Plot(plot_losses)

Plotting results


Plotting is done with matplotlib, using the array of loss values plot_losses saved while training.

In [18]:
def show_Plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


Evaluation

Evaluation is mostly the same as training, but there are no targets so we simply feed the decoder’s predictions back to itself for each step. Every time it predicts a word we add it to the output string, and if it predicts the EOS token we stop there. We also store the decoder’s attention outputs for display later.

In [51]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./runs/nlp_seq_seq')

In [60]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_Sentence(input_language, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        writer.add_graph(encoder, (input_tensor[0], encoder_hidden))
        writer.close()

        

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        writer.add_graph(decoder, (decoder_input, decoder_hidden, encoder_outputs))
        writer.close()
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
           
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_language.index_to_word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [61]:
def evaluate_Randomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


Training and Evaluating

With all these helper functions in place (it looks like extra work, but it makes it easier to run multiple experiments) we can actually initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small dataset we can use relatively small networks of 256 hidden nodes and a single GRU layer. After about 40 minutes on a MacBook CPU we’ll get some reasonable results.



In [21]:
hidden_size = 256
encoder1 = EncoderRNN(input_language.num_words, hidden_size).to(device)
attn_decoder1 = AttentionDecoderRNN(hidden_size, output_language.num_words, dropout_p=0.1).to(device)

train_Iters(encoder1, attn_decoder1, 75000, print_every=5000)

5m 49s (- 81m 28s) (5000 6%) 2.8677
11m 22s (- 73m 53s) (10000 13%) 2.2908
16m 49s (- 67m 16s) (15000 20%) 1.9334
22m 14s (- 61m 10s) (20000 26%) 1.7582
27m 43s (- 55m 26s) (25000 33%) 1.5405
33m 6s (- 49m 39s) (30000 40%) 1.3675
38m 41s (- 44m 13s) (35000 46%) 1.2223
44m 2s (- 38m 32s) (40000 53%) 1.0970
49m 30s (- 33m 0s) (45000 60%) 1.0157
54m 59s (- 27m 29s) (50000 66%) 0.9003
60m 27s (- 21m 59s) (55000 73%) 0.8039
65m 56s (- 16m 29s) (60000 80%) 0.7315
71m 27s (- 10m 59s) (65000 86%) 0.6986
77m 0s (- 5m 30s) (70000 93%) 0.6184
82m 46s (- 0m 0s) (75000 100%) 0.5517


Visualizing Attention

A useful property of the attention mechanism is its highly interpretable outputs. Because it is used to weight specific encoder outputs of the input sequence, we can imagine looking where the network is focused most at each time step.

You could simply run plt.matshow(attentions) to see attention output displayed as a matrix, with the columns being input steps and rows being output steps:

In [65]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

In [66]:
def show_Attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluate_And_Show_Attention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    show_Attention(input_sentence, output_words, attentions)


evaluate_And_Show_Attention("elle a cinq ans de moins que moi .")

evaluate_And_Show_Attention("elle est trop petit .")

evaluate_And_Show_Attention("je ne crains pas de mourir .")

evaluate_And_Show_Attention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she is five years younger than me . <EOS>
input = elle est trop petit .
output = she s too short . <EOS>
input = je ne crains pas de mourir .
output = i m not scared of dying . <EOS>
input = c est un jeune directeur plein de talent .
output = he s a talented young director . <EOS>


In [32]:
torch.save(encoder1.state_dict(), './model/encoder.pth')

In [33]:
torch.save(attn_decoder1.state_dict(), './model/attention_decoder.pth')